## benchmarking with the speedglm implementation
revised 03 August 2017, by [Nima Hejazi](http://nimahejazi.org)

The purpose of this notebook is to benchmark the performance of the `survtmle` package using the standard `glm` implementation, on simulated data sets of varying sample sizes ($n = 100, 1000, 5000$). This is one of two notebooks meant to compare the performance of `glm` against that of `speedglm`.

### preliminaries

In [1]:
# preliminaries
library(microbenchmark)

# set seed and constants
set.seed(341796)
t_0 <- 15

In [2]:
## get correct version of `survtmle`
if ("survtmle" %in% installed.packages()) {
    remove.packages("survtmle")
}
suppressMessages(devtools::install_github("benkeser/survtmle", ref = "speedglm"))
library(survtmle)

Removing package from ‘/Users/nimahejazi/.Rlibrary’
(as ‘lib’ is unspecified)
survtmle: Targeted Learning for Survival Analysis
Version: 1.0.1.2


## <u>Example 1: simple simulated data set</u>
This is a rather trivial example wherein the simulated data set contains few covariates.

### case 1: _n = 100 (trivial example)_

In [3]:
# simulate data
n <- 100
W <- data.frame(W1 = runif(n), W2 = rbinom(n, 1, 0.5))
A <- rbinom(n, 1, 0.5)
T <- rgeom(n,plogis(-4 + W$W1 * W$W2 - A)) + 1
C <- rgeom(n, plogis(-6 + W$W1)) + 1
ftime <- pmin(T, C)
ftype <- as.numeric(ftime == T)

In [5]:
system.time(
    fit <- survtmle(ftime = ftime, ftype = ftype, trt = A, adjustVars = W,
                    glm.trt = "1", glm.ftime = "I(W1*W2) + trt + t",
                    glm.ctime = "W1 + t", method = "hazard", t0 = t_0)
)

'speedglm' ran into an error in estimateTreatment ... using 'glm' instead.


   user  system elapsed 
  1.068   0.028   1.206 

In [6]:
suppressMessages(
    m1 <- microbenchmark(unit = "s",
        fit <- survtmle(ftime = ftime, ftype = ftype, trt = A, adjustVars = W,
                        glm.trt = "1", glm.ftime = "I(W1*W2) + trt + t",
                        glm.ctime = "W1 + t", method = "hazard", t0 = t_0)
    )
)
summary(m1)

expr,min,lq,mean,median,uq,max,neval
"fit <- survtmle(ftime = ftime, ftype = ftype, trt = A, adjustVars = W, glm.trt = ""1"", glm.ftime = ""I(W1*W2) + trt + t"", glm.ctime = ""W1 + t"", method = ""hazard"", t0 = t_0)",0.2023905,0.3553329,0.4939564,0.4316623,0.5383586,1.740011,100


This trivial example is merely provided for comparison against the following cases with larger sample sizes.

### case 2: _n = 1000_

In [10]:
# simulate data
n <- 1000
W <- data.frame(W1 = runif(n), W2 = rbinom(n, 1, 0.5))
A <- rbinom(n, 1, 0.5)
T <- rgeom(n,plogis(-4 + W$W1 * W$W2 - A)) + 1
C <- rgeom(n, plogis(-6 + W$W1)) + 1
ftime <- pmin(T, C)
ftype <- as.numeric(ftime == T)

In [11]:
system.time(
    fit <- survtmle(ftime = ftime, ftype = ftype, trt = A, adjustVars = W,
                    glm.trt = "1", glm.ftime = "I(W1*W2) + trt + t",
                    glm.ctime = "W1 + t", method = "hazard", t0 = t_0)
)

'speedglm' ran into an error in estimateHazards ... using 'glm' instead.


   user  system elapsed 
  2.966   0.427   4.450 

In [12]:
suppressMessages(
    m2 <- microbenchmark(unit = "s",
        fit <- survtmle(ftime = ftime, ftype = ftype, trt = A, adjustVars = W,
                        glm.trt = "1", glm.ftime = "I(W1*W2) + trt + t",
                        glm.ctime = "W1 + t", method = "hazard", t0 = t_0)
    )
)
summary(m2)

expr,min,lq,mean,median,uq,max,neval
"fit <- survtmle(ftime = ftime, ftype = ftype, trt = A, adjustVars = W, glm.trt = ""1"", glm.ftime = ""I(W1*W2) + trt + t"", glm.ctime = ""W1 + t"", method = ""hazard"", t0 = t_0)",3.659519,4.316093,5.54298,5.176611,6.436896,11.74602,100


...

### case 3: _n = 5000_

In [13]:
# simulate data
n <- 5000
W <- data.frame(W1 = runif(n), W2 = rbinom(n, 1, 0.5))
A <- rbinom(n, 1, 0.5)
T <- rgeom(n,plogis(-4 + W$W1 * W$W2 - A)) + 1
C <- rgeom(n, plogis(-6 + W$W1)) + 1
ftime <- pmin(T, C)
ftype <- as.numeric(ftime == T)

In [14]:
system.time(
    fit <- survtmle(ftime = ftime, ftype = ftype, trt = A, adjustVars = W,
                    glm.trt = "1", glm.ftime = "I(W1*W2) + trt + t",
                    glm.ctime = "W1 + t", method = "hazard", t0 = t_0)
)

'speedglm' ran into an error in estimateHazards ... using 'glm' instead.


   user  system elapsed 
 20.423   2.505  36.501 

In [49]:
#m3 <- microbenchmark(unit = "s",
#    fit <- survtmle(ftime = ftime, ftype = ftype, trt = A, adjustVars = W,
#                    glm.trt = "1", glm.ftime = "I(W1*W2) + trt + t",
#                    glm.ctime = "W1 + t", method = "hazard", t0 = t_0)
#)
#summary(m3)

This case takes too long to benchmark properly.

## <u>Example 2: a "more real" simulated data set</u>
This is a more interesting example wherein the simulated data set contains a larger number of covariates, which might be interesting in real-world / practical applications.

In [6]:
# functions for this simulation
get.ftimeForm <- function(trt, site){
	form <- "-1"
	for(i in unique(trt)){
		for(s in unique(site)){
			form <- c(form, 
			  paste0("I(trt==",i,"& site == ",s," & t==",
			         unique(ftime[ftype>0 & trt==i & site == s]),")",
			         collapse="+"))
		}
	}
	return(paste(form,collapse="+"))
}

get.ctimeForm <- function(trt, site){
	form <- "-1"
	for(i in unique(trt)){
		for(s in unique(site)){
			form <- c(form, 
			  paste0("I(trt==",i,"& site == ",s," & t==",
			         unique(ftime[ftype==0 & trt==i & site == s]),")",
			         collapse="+"))
		}
	}
	return(paste(form,collapse="+"))
}

### case 1: _n = 100 (trivial example)_

In [7]:
# simulate data
n <- 100
trt <- rbinom(n, 1, 0.5)

# e.g., study site
adjustVars <- data.frame(site = (rbinom(n,1,0.5) + 1))
ftime <- round(1 + runif(n, 1, 350) - trt + adjustVars$site)
ftype <- round(runif(n, 0, 1))

glm.ftime <- get.ftimeForm(trt = trt, site = adjustVars$site)
glm.ctime <- get.ctimeForm(trt = trt, site = adjustVars$site)

In [8]:
system.time(
    fit <- survtmle(ftime = ftime, ftype = ftype, trt = trt, adjustVars = adjustVars,
                    glm.trt = "1", glm.ftime = glm.ftime, glm.ctime = glm.ctime,
                    method = "hazard", t0 = t_0)
)

'speedglm' ran into an error in estimateTreatment ... using 'glm' instead.


   user  system elapsed 
  1.571   0.132   1.897 

In [9]:
suppressMessages(
    m4 <- microbenchmark(unit = "s", times = 10,
        fit <- survtmle(ftime = ftime, ftype = ftype, trt = trt, adjustVars = adjustVars,
                        glm.trt = "1", glm.ftime = glm.ftime, glm.ctime = glm.ctime,
                        method = "hazard", t0 = t_0)
    )
)
summary(m4)

expr,min,lq,mean,median,uq,max,neval
"fit <- survtmle(ftime = ftime, ftype = ftype, trt = trt, adjustVars = adjustVars, glm.trt = ""1"", glm.ftime = glm.ftime, glm.ctime = glm.ctime, method = ""hazard"", t0 = t_0)",1.423693,1.513326,1.697925,1.673614,1.772479,2.097713,10


This trivial example is merely provided for comparison against the following cases with larger sample sizes.

### case 2: _n = 1000_

In [10]:
# simulate data
n <- 1000
trt <- rbinom(n, 1, 0.5)

# e.g., study site
adjustVars <- data.frame(site = (rbinom(n,1,0.5) + 1))
ftime <- round(1 + runif(n, 1, 350) - trt + adjustVars$site)
ftype <- round(runif(n, 0, 1))

glm.ftime <- get.ftimeForm(trt = trt, site = adjustVars$site)
glm.ctime <- get.ctimeForm(trt = trt, site = adjustVars$site)

In [11]:
system.time(
    fit <- survtmle(ftime = ftime, ftype = ftype, trt = trt, adjustVars = adjustVars,
                    glm.trt = "1", glm.ftime = glm.ftime, glm.ctime = glm.ctime,
                    method = "hazard", t0 = t_0)
)

'speedglm' ran into an error in estimateTreatment ... using 'glm' instead.


   user  system elapsed 
314.564 196.807 988.427 

In [ ]:
suppressMessages(
    m5 <- microbenchmark(unit = "s", times = 10,
        fit <- survtmle(ftime = ftime, ftype = ftype, trt = trt, adjustVars = adjustVars,
                        glm.trt = "1", glm.ftime = glm.ftime, glm.ctime = glm.ctime,
                        method = "hazard", t0 = t_0)
    )
)
summary(m5)

commentary here...

### case 3: _n = 5000_

In [ ]:
# simulate data
#n <- 5000
#trt <- rbinom(n, 1, 0.5)

# e.g., study site
#adjustVars <- data.frame(site = (rbinom(n,1,0.5) + 1))
#ftime <- round(1 + runif(n, 1, 350) - trt + adjustVars$site)
#ftype <- round(runif(n, 0, 1))

#glm.ftime <- get.ftimeForm(trt = trt, site = adjustVars$site)
#glm.ctime <- get.ctimeForm(trt = trt, site = adjustVars$site)

In [ ]:
#system.time(
#    fit <- survtmle(ftime = ftime, ftype = ftype, trt = trt, adjustVars = adjustVars,
#                    glm.trt = "1", glm.ftime = glm.ftime, glm.ctime = glm.ctime,
#                    method = "hazard", t0 = t_0)
#)

In [ ]:
#m6 <- microbenchmark(unit = "s",
#    fit <- survtmle(ftime = ftime, ftype = ftype, trt = trt, adjustVars = adjustVars,
#                    glm.trt = "1", glm.ftime = glm.ftime, glm.ctime = glm.ctime,
#                    method = "hazard", t0 = t_0)
#)
#summary(m6)

As in the previous example, this case takes too long to benchmark properly.

In [10]:
fit <- survtmle(ftime = ftime, ftype = ftype, trt = trt, adjustVars = adjustVars,
                    glm.trt = "1", glm.ftime = glm.ftime, glm.ctime = glm.ctime,
                    method = "hazard", t0 = t_0)
traceback()

'speedglm' ran into an error in estimateCensoring ... using 'glm' instead.
'speedglm' ran into an error in estimateHazards ... using 'glm' instead.


No traceback available 


No traceback available 
